# Este cuadernillo sirve como notas para el uso de los comandos de PostgreSQL

## Al igual que con el curso de PowerBi, la idea es trabajar con datos de trabajos anteriores y repasar algunos conceptos que se han visto durante el curso.

## Como un extra, me interesa poder aplicar lenguaje de programación de Python y utilizar PowerBi para la visualizacion del contenido.


### Referencias:
 - postgresqltutorial.com/postgresql-python/
 
Lo primero será descargar la biblioteca:
- pip install psycopg2

**Nota**: Si adicionalmente se quiere utilizar jupyter y postgreSQL juntos, entonces recomiendo instalarlo tanto en el entorno de anaconda, como en la terminal del computador mismo.



### Para establecer la coneccion entre PostgreSQL y Python

In [69]:
# para establecer la coneccion con la base de datos
import psycopg2a
conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')


### Observemos los datos que vamos a importar a PostgreSQL

In [16]:
# lo siguiente que podemos intentar es hacer la siguiente tabla en postgreSQL
import pandas as pd

ruta = 'https://raw.githubusercontent.com/raaraya1/University-Projects/main/Logistics/Datos/DCs.csv'
archivo = pd.read_csv(ruta, sep=';')
df = pd.DataFrame(archivo)
df

,DCID,DCLocationLatitude,DCLocationLongitude,DCTransportationCapacityLiters,HandlingCostPerLiter
0,DC001,-1.167.114.585,-8.322.489.278,88,48
1,DC002,-8.195.947.549,7.291.626.950,37,96
2,DC003,-1.422.731.698,6.443.633.751,58,82
3,DC004,4.358.105.746,8.986.685.836,48,29
4,DC005,8.411.115.264,-3.458.885.675,93,49


In [70]:
# de esta manera tendremos que hacer una tabla, poniendo como primary key el 'DCID'
# primero hacemos un cursor
cur = conn.cursor()

# luego escribimos el comando que queremos ejecutar
comando = 'create table dc (DCID varchar(20) primary key, DCLocationLatitude varchar(20), DCLocationLongitude varchar(20), DCTransportationCapacityLiters int, HandlingCostPerLiter int)'
                
# luego tenemos que executar el cursor
cur.execute(comando)

# por ultimo cerramos el cursor
cur.close()

# para guardar los cambios
conn.commit()

### Para insertar los datos 

In [76]:
# ahora vamos a insertar los datos
# primero originemos los comandos que queremos ejecutar

comandos = []
for i in range(len(df)):    
    id = df['DCID'][i]
    latitud = df['DCLocationLatitude'][i]
    longitud = df['DCLocationLongitude'][i]
    capacidad = df['DCTransportationCapacityLiters'][i]
    costo = df['HandlingCostPerLiter'][i]
    comando = "insert into dc values ('{}', '{}', '{}', {}, {})".format(id, str(latitud), str(longitud), capacidad, costo)    
    comandos.append(comando)
for i in comandos:
    print(i)



insert into dc values ('DC001', '-1.167.114.585', '-8.322.489.278', 88, 48)
insert into dc values ('DC002', '-8.195.947.549', '7.291.626.950', 37, 96)
insert into dc values ('DC003', '-1.422.731.698', '6.443.633.751', 58, 82)
insert into dc values ('DC004', '4.358.105.746', '8.986.685.836', 48, 29)
insert into dc values ('DC005', '8.411.115.264', '-3.458.885.675', 93, 49)


In [77]:
# volvemos a repetir los pasos para ejecutar un comando en postgreSQL

# establecer la coneccion con la base de datos    
conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')

# crear un cursor
cur1 = conn.cursor()

# ejecutamos los comandos
for comando in comandos:
    cur1.execute(comando)

# cerramos el cursor
cur1.close()

# guardamos los cambios
conn.commit()

### Verficar que los datos fueron importados correctamente

**Opcion 1:** Recorriendo las filas

In [82]:
# verificamos que los cambios se hallan efectuado
# establecer la coneccion con la base de datos    
conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')
cur = conn.cursor()

# seleccionar la tabla
cur.execute('select * from dc')

# recorrer las filas
row = cur.fetchone()
while row is not None:
    print(row)
    row = cur.fetchone()   


# cerrar
cur.close()
conn.close()


('DC001', '-1.167.114.585', '-8.322.489.278', 88, 48)
('DC002', '-8.195.947.549', '7.291.626.950', 37, 96)
('DC003', '-1.422.731.698', '6.443.633.751', 58, 82)
('DC004', '4.358.105.746', '8.986.685.836', 48, 29)
('DC005', '8.411.115.264', '-3.458.885.675', 93, 49)


**Opcion 2:** Mostrando toda la tabla

In [84]:
# tambien se pueden obtener todas las filas (sin tener que recorrerlas)

# establecer la coneccion con la base de datos    
conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')
cur = conn.cursor()

# seleccionar la tabla
cur.execute('select * from dc')
    
# obtener todas las filas
rows = cur.fetchall()
df1 = pd.DataFrame(rows)

# cerrar
cur.close()
conn.close()

df1

,0,1,2,3,4
0,DC001,-1.167.114.585,-8.322.489.278,88,48
1,DC002,-8.195.947.549,7.291.626.950,37,96
2,DC003,-1.422.731.698,6.443.633.751,58,82
3,DC004,4.358.105.746,8.986.685.836,48,29
4,DC005,8.411.115.264,-3.458.885.675,93,49


### Llegados a este  punto, puede ser conveniente establecer una funcion para asi no estar constantemente estableciendo la conexion con el programa y creando un cursor para su interaccion.


In [16]:
## la forma que esta interactuara es recibiendo un string como parametro 

# importar biblioteca
import psycopg2

# creamos la funcion
def ejecutar(comando):
    # establecer la conexion
    conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')
    
    # creamos el cursor
    cur = conn.cursor()
    
    # ejecutar comando
    cur.execute(comando)

    # guardamos los cambios
    conn.commit()
    
    # cerrar
    cur.close()
    conn.close()
    



### Ahora probemos la funcion con algunos de los comandos que ejecutamos anteriormente

In [17]:
# ahora vamos a crear la tabla 'customers'

import pandas as pd

ruta = 'https://raw.githubusercontent.com/raaraya1/University-Projects/main/Logistics/Datos/customers.csv'
archivo = pd.read_csv(ruta, sep = ';')
df2 = pd.DataFrame(archivo)
df2

# comando para crear la tabla
comando_crear_tabla = '''create table customers(CustomerID varchar(20) primary key, 
CustomerLocationLatitude varchar(20), CustomerLocationLongitude varchar(20)); 
'''

# ejecutamos el comando
ejecutar(comando_crear_tabla)
    

### Ahora vamos a insertarle los datos

In [38]:
# insertar los datos

nombre_columnas = [columna for columna in df2]

# ejemplo de comando
#comando = "inster into customers values ('{}', '{}', '{}')".format(df2[nombre_columnas[0]][0], df2[nombre_columnas[1]][0], df2[nombre_columnas[2]][0])    
#print(comando)

for fila in range(len(df2)):    
    comando = "insert into customers values ('{}', '{}', '{}')".format(df2[nombre_columnas[0]][fila], df2[nombre_columnas[1]][fila], df2[nombre_columnas[2]][fila])    
    ejecutar(comando)   
    

### Revisemos que los datos se hallan guardado

In [39]:
# esta funcion nos puede ayudar a mostrar mas rapidamente los datos

def mostrar_tabla(comando):
    # establecer la conexion
    conn = psycopg2.connect('dbname=curso_postgresql user=Tester password=123456')
    
    # creamos el cursor
    cur = conn.cursor()
    
    # ejecutar comando
    cur.execute(comando)
    
    # obtener todas las filas
    rows = cur.fetchall()
    df = pd.DataFrame(rows)

    # guardamos los cambios
    conn.commit()
    
    # cerrar
    cur.close()
    conn.close()
    
    return df

In [40]:
comando = '''select * from customers'''
customers = mostrar_tabla(comando)
customers

,0,1,2
0,Customer001,-6.974.150.410,0.572.172.540
1,Customer002,-1.774.631.895,0.771.807.615
2,Customer003,6.741.158.818,7.906.606.020
3,Customer004,-7.191.233.417,-2.539.319.369
4,Customer005,6.844.101.913,-6.249.004.925
5,Customer006,-5.887.849.020,-2.207.052.049
6,Customer007,2.739.095.208,-4.938.065.519
